In [3]:
import sagemaker
import boto3
import csv
import numpy as np
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)
s3 = boto3.Session().client(service_name="s3", region_name=region)

# Combine Data

In [4]:
!aws s3 cp 's3://ads-508-team4/olist/csv/cust/olist_customers_dataset.csv' ./data-clarify/
!aws s3 cp 's3://ads-508-team4/olist/csv/order_items/olist_order_items_dataset.csv' ./data-clarify/
!aws s3 cp 's3://ads-508-team4/olist/csv/order/olist_orders_dataset.csv' ./data-clarify/
!aws s3 cp 's3://ads-508-team4/olist/csv/products/olist_products_dataset.csv' ./data-clarify/

download: s3://ads-508-team4/olist/csv/cust/olist_customers_dataset.csv to data-clarify/olist_customers_dataset.csv
download: s3://ads-508-team4/olist/csv/order_items/olist_order_items_dataset.csv to data-clarify/olist_order_items_dataset.csv
download: s3://ads-508-team4/olist/csv/order/olist_orders_dataset.csv to data-clarify/olist_orders_dataset.csv
download: s3://ads-508-team4/olist/csv/products/olist_products_dataset.csv to data-clarify/olist_products_dataset.csv


### 1.Customer dataset

In [5]:
df_cust = pd.read_csv(
    "./data-clarify/olist_customers_dataset.csv",
    delimiter=",",
    quoting=csv.QUOTE_NONE
)

In [6]:
df_cust.columns=df_cust.columns.str.replace('"','')
df_cust['customer_id'] = df_cust['customer_id'].str.replace('"','')
df_cust['customer_unique_id'] = df_cust['customer_unique_id'].str.replace('"','')
df_cust['customer_zip_code_prefix'] = df_cust['customer_zip_code_prefix'].str.replace('"','')

In [7]:
df_cust.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


### 2. Order items dataset

In [8]:
order_items = pd.read_csv(
    "./data-clarify/olist_order_items_dataset.csv",
    delimiter=",",
    quoting=csv.QUOTE_NONE
)

In [9]:
order_items.columns=order_items.columns.str.replace('"','')
order_items['order_id'] = order_items['order_id'].str.replace('"','')
order_items['product_id'] = order_items['product_id'].str.replace('"','')
order_items['seller_id'] = order_items['seller_id'].str.replace('"','')

In [10]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


### 3. Order dataset

In [11]:
order = pd.read_csv(
    "./data-clarify/olist_orders_dataset.csv",
    delimiter=",",
    quoting=csv.QUOTE_NONE
)

In [12]:
order.columns=order.columns.str.replace('"','')
order['order_id'] = order['order_id'].str.replace('"','')
order['customer_id'] = order['customer_id'].str.replace('"','')

In [13]:
order.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


### 4. product

In [14]:
prod = pd.read_csv(
    "./data-clarify/olist_products_dataset.csv",
    delimiter=",",
    quoting=csv.QUOTE_NONE
)

In [15]:
prod.columns=prod.columns.str.replace('"','')
prod['product_id'] = prod['product_id'].str.replace('"','')

In [16]:
prod.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


## Merge datasets

In [17]:
order_cust = order.merge(df_cust, how='left', on='customer_id' )
order_cust_item = order_cust.merge(order_items, how='left', on='order_id')
master = order_cust_item.merge(prod, how='left', on='product_id')

In [52]:
master.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,price_grp,agro_industry_and_commerce
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,03149,...,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,Q1,0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,...,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,Q3,0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,Q4,0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,...,pet_shop,59.0,468.0,3.0,450.0,30.0,10.0,20.0,Q2,0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,09195,...,papelaria,38.0,316.0,4.0,250.0,51.0,15.0,15.0,Q1,0


# Transform product categoreis and translate to English

In [119]:
master['agro_industry_and_commerce'] = np.where(master['product_category_name'].isin(['agro_industria_e_comercio']), 1, 0)
master['foods'] = np.where(master['product_category_name'].isin(['alimentos','alimentos_bebidas','la_cuisine']), 1 ,0)
master['arts'] = np.where(master['product_category_name'].isin(['artes','artes_e_artesanato']),1 ,0)
master['party'] = np.where(master['product_category_name'].isin(['artigos_de_festas','artigos_de_natal']), 1 ,0)
master['audio'] = np.where(master['product_category_name'].isin(['audio']), 1 ,0)
master['automotive'] = np.where(master['product_category_name'].isin(['automotivo']), 1 ,0)
master['babies'] = np.where(master['product_category_name'].isin(['bebes','fraldas_higiene']), 1 ,0)
master['drinks'] = np.where(master['product_category_name'].isin(['bebidas']), 1 ,0)
master['beauty_health'] = np.where(master['product_category_name'].isin(['beleza_saude']), 1 ,0)
master['toys'] = np.where(master['product_category_name'].isin(['brinquedos']), 1 ,0)

master['bed_table_bath'] = np.where(master['product_category_name'].isin(['cama_mesa_banho']), 1 ,0)
master['home'] = np.where(master['product_category_name'].isin(['casa_conforto','casa_conforto_2','casa_construcao', 'utilidades_domesticas']), 1 ,0)
master['music_cds_dvds'] = np.where(master['product_category_name'].isin(['cds_dvds_musicais']), 1 ,0)
master['cine_photo'] = np.where(master['product_category_name'].isin(['cine_foto']), 1 ,0)
master['air_conditioning'] = np.where(master['product_category_name'].isin(['climatizacao']), 1 ,0)
master['game_consoles'] = np.where(master['product_category_name'].isin(['consoles_games']), 1 ,0)
master['construction'] = np.where(master['product_category_name'].isin(['construcao_ferramentas_construcao','construcao_ferramentas_ferramentas','construcao_ferramentas_iluminacao','construcao_ferramentas_jardim','construcao_ferramentas_seguranca']), 1 ,0)
master['cool_stuff'] = np.where(master['product_category_name'].isin(['cool_stuff','seguros_e_servicos','sinalizacao_e_seguranca']), 1 ,0)
master['blu_ray_dvds'] = np.where(master['product_category_name'].isin(['dvds_blu_ray']), 1 ,0)
master['electronics'] = np.where(master['product_category_name'].isin(['eletrodomesticos','eletrodomesticos_2', 'eletronicos', 'eletroportateis']), 1 ,0)

master['sport_leisure'] = np.where(master['product_category_name'].isin(['esporte_lazer']), 1 ,0)
master['fashion'] = np.where(master['product_category_name'].isin(['fashion_bolsas_e_acessorios','fashion_calcados','fashion_esporte','fashion_roupa_feminina','fashion_roupa_infanto_juvenil','fashion_roupa_masculina','fashion_underwear_e_moda_praia']), 1 ,0)
master['garden'] = np.where(master['product_category_name'].isin(['ferramentas_jardim','flores']), 1 ,0)
master['industry'] = np.where(master['product_category_name'].isin(['industria_comercio_e_negocios']), 1 ,0)
master['computer_accessories'] = np.where(master['product_category_name'].isin(['informatica_acessorios']), 1 ,0)
master['musical_instruments'] = np.where(master['product_category_name'].isin(['instrumentos_musicais']), 1 ,0)
master['books'] = np.where(master['product_category_name'].isin(['livros_importados','livros_interesse_geral','livros_tecnicos']), 1 ,0)
master['accessory_bags'] = np.where(master['product_category_name'].isin(['malas_acessorios']), 1 ,0)
master['market_place'] = np.where(master['product_category_name'].isin(['market_place']), 1 ,0)
master['furniture'] = np.where(master['product_category_name'].isin(['moveis_colchao_e_estofado','moveis_cozinha_area_de_servico_jantar_e_jardim','moveis_decoracao','moveis_escritorio','moveis_quarto','moveis_sala']), 1 ,0)

master['song'] = np.where(master['product_category_name'].isin(['musica']), 1 ,0)
master['stationery'] = np.where(master['product_category_name'].isin(['papelaria']), 1 ,0)
master['computer'] = np.where(master['product_category_name'].isin(['pc_gamer','pcs', 'portateis_casa_forno_e_cafe','portateis_cozinha_e_preparadores_de_alimentos']), 1 ,0)
master['perfume'] = np.where(master['product_category_name'].isin(['perfumaria']), 1 ,0)
master['pet'] = np.where(master['product_category_name'].isin(['pet_shop']), 1 ,0)
master['gifts_watches'] = np.where(master['product_category_name'].isin(['relogios_presentes']), 1 ,0)
master['insurance_and_services'] = np.where(master['product_category_name'].isin(['seguros_e_servicos']), 1 ,0)
master['phone'] = np.where(master['product_category_name'].isin(['telefonia','telefonia_fixa']), 1 ,0)

In [121]:
master.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,furniture,song,stationery,pcs,perfume,pet,computer,gifts_watches,insurance_and_services,phone
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,03149,...,0,0,0,0,0,0,0,0,0,0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,...,0,0,0,0,1,0,0,0,0,0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,0,0,0,0,0,0,0,0,0,0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,...,0,0,0,0,0,1,0,0,0,0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,09195,...,0,0,1,0,0,0,0,0,0,0


# Transform dataset to customer level

### Aggreate orders

In [154]:
cust_master = master.groupby(['customer_unique_id']).agg({
'order_id'                       : 'count',
'price'                          : 'sum',
'freight_value'                  : 'sum',
'product_weight_g'               : 'sum',
'agro_industry_and_commerce'     : 'sum',
'foods'                          : 'sum',
'arts'                           : 'sum',
'articles'                       : 'sum',
'audio'                          : 'sum', 
'automotive'                     : 'sum', 
'babies'                         : 'sum', 
'drinks'                         : 'sum',  
'beauty_health'                  : 'sum',
'toys'                           : 'sum',
'bed table bath'                 : 'sum',
'bed_table_bath'                 : 'sum',
'home'                           : 'sum', 
'music_cds_dvds'                 : 'sum',
'cine_photo'                     : 'sum',
'air_conditioning'               : 'sum',
'game_consoles'                  : 'sum', 
'construction'                   : 'sum',
'cool_stuff'                     : 'sum',
'blu_ray_dvds'                   : 'sum',
'home_appliances'                : 'sum',
'electronics'                    : 'sum',
'sport leisure'                  : 'sum',
'fashion'                        : 'sum', 
'garden'                         : 'sum',
'sport_leisure'                  : 'sum', 
'industry'                       : 'sum',
'musical_instruments'            : 'sum',
'computer_accessories'           : 'sum',
'party'                          : 'sum', 
'books'                          : 'sum',
'accessory_bags'                 : 'sum',
'market_place'                   : 'sum', 
'furniture'                      : 'sum', 
'song'                           : 'sum', 
'stationery'                     : 'sum',
'pcs'                            : 'sum', 
'perfume'                        : 'sum',
'pet'                            : 'sum', 
'computer'                       : 'sum',
'gifts_watches'                  : 'sum',
'insurance_and_services'         : 'sum',
'phone'                          : 'sum'
}).reset_index()

In [167]:
print(cust_master.shape)
cust_master.head()

(96096, 48)


,customer_unique_id,order_cnt,total_spend,total_shipping,total_prod_weight_g,agro_industry_and_commerce,foods,arts,articles,audio,...,furniture,song,stationery,pcs,perfume,pet,computer,gifts_watches,insurance_and_services,phone
0,0000366f3b9a7992bf8c76cfdf3221e2,1,129.90,12.00,1500.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,18.90,8.29,375.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0000f46a3911fa3c0805444483337064,1,69.00,17.22,1500.0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0000f6ccb0745a6a4b88665a16c9f078,1,25.99,17.63,150.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0004aac84e0df4da2b147fca70cf8255,1,180.00,16.89,6050.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [158]:
cust_master.columns = ['customer_unique_id', 'order_cnt',
                       'total_spend', 'total_shipping', 'total_prod_weight_g',
                       'agro_industry_and_commerce', 'foods', 'arts', 'articles', 'audio',
                       'automotive', 'babies', 'drinks', 'beauty_health', 'toys',
                       'bed table bath', 'bed_table_bath', 'home', 'music_cds_dvds',
                       'cine_photo', 'air_conditioning', 'game_consoles', 'construction',
                       'cool_stuff', 'blu_ray_dvds', 'home_appliances', 'electronics',
                       'sport leisure', 'fashion', 'garden', 'sport_leisure', 'industry',
                       'musical_instruments', 'computer_accessories', 'party', 'books',
                       'accessory_bags', 'market_place', 'furniture', 'song', 'stationery',
                       'pcs', 'perfume', 'pet', 'computer', 'gifts_watches',
                       'insurance_and_services', 'phone']

### Latest state and city infomation 

In [170]:
demo = master.sort_values(['order_purchase_timestamp'], ascending=False).groupby(['customer_unique_id']).head(1)
demo = demo[['customer_unique_id', 'customer_city', 'customer_state' ]]

In [171]:
print(demo.shape)
demo.head()

(96096, 3)


,customer_unique_id,customer_city,customer_state
69370,87ab9fec999db8bd5774917de3cdf01c,sorocaba,SP
77917,262e1f1e26e92e86375f86840b4ffd63,picos,PI
36374,af5454198a97379394cacf676e1e96cb,registro,SP
100823,634420a0ea42302205032ed44ac7fccc,pirai,RJ
57391,9bb92bebd4cb7511e1a02d5e50bc4655,guarulhos,SP


### Merge state and city to the master dataset

In [172]:
cust_master = cust_master.merge(demo, how='left', on='customer_unique_id' )

In [174]:
cust_master.head()cust_master

,customer_unique_id,order_cnt,total_spend,total_shipping,total_prod_weight_g,agro_industry_and_commerce,foods,arts,articles,audio,...,stationery,pcs,perfume,pet,computer,gifts_watches,insurance_and_services,phone,customer_city,customer_state
0,0000366f3b9a7992bf8c76cfdf3221e2,1,129.90,12.00,1500.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,cajamar,SP
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,18.90,8.29,375.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,osasco,SP
2,0000f46a3911fa3c0805444483337064,1,69.00,17.22,1500.0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,sao jose,SC
3,0000f6ccb0745a6a4b88665a16c9f078,1,25.99,17.63,150.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,belem,PA
4,0004aac84e0df4da2b147fca70cf8255,1,180.00,16.89,6050.0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,sorocaba,SP


# Remove outliers

In [199]:
from scipy import stats
cust_remove_out = cust_master[(np.abs(stats.zscore(cust_master['total_spend'])) < 3)]
cust_remove_out.shape

(94412, 51)

In [200]:
cust_remove_out['total_spend'].describe()

count    94412.000000
mean       120.498438
std        117.599781
min          0.000000
25%         45.900000
50%         86.895000
75%        149.900000
max        792.000000
Name: total_spend, dtype: float64

# Set target variable

In [201]:
cust_master['spending_grp'] = pd.cut(cust_master['total_spend'], 
                                    [0, 45.9, 86.895, 149.9, np.inf],
                                    labels=["Normal", "Average", "Good", "Excellent"])

In [202]:
cust_master['spending_grp'].value_counts()

Excellent    24914
Good         23976
Average      23302
Normal       23228
Name: spending_grp, dtype: int64

# Split datasets into train, test, and validate

* 60% train
* 20% validate
* 20% test

In [203]:
from sklearn.model_selection import train_test_split

In [204]:
train, validate, test = np.split(cust_master.sample(frac=1, random_state=42), 
                        [int(.6*len(cust_master)), int(.8*len(cust_master))])

In [217]:
print(train.shape)
print(validate.shape)
print(test.shape)

(57657, 52)
(19219, 52)
(19220, 52)
